# Index

1. [Packages](#packages)
2. [Guardar los datos estandarizados](#guardar-los-datos-estandarizados)

# Packages

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mlparadetectarfraudes.data import data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold #StratifiedKFold para modelo distinto a RandomForest
from sklearn.metrics import confusion_matrix, classification_report, f1_score #accuracy_score
from sklearn.preprocessing import StandardScaler ,OneHotEncoder, scale

from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline, make_pipeline # Corrected from Pipelines
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib. import style
style.use("ggplot") or plt.style.use("ggplot")
import statsmodels.api as sm
from Ipython.display import display

In [ ]:
# Separate features and target
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

# Identify column types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessing pipelines
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #Reemplazar valores faltantes con la mediana
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), #Reemplazar valores faltantes con  los valores más comunes
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first')) # Use variables dummies
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X_processed = preprocessor.fit_transform(X)
X_processed.shape()



In [8]:
column_names = preprocessor.get_feature_names_out()
X_processed_df = pd.DataFrame(X_processed, columns=column_names)
X_processed_df.head()
X_processed_df.shape()


TypeError: 'tuple' object is not callable

In [ ]:
# Generate subplots for each variable.
fig, axs = plt.subplots(nrows=6, ncols=3, figsize=(15, 20))
axs = axs.flatten()

# Generate a boxplot for each variable.
for i, var in enumerate(data.columns):  # We iterated directly over the DataFrame’s columns.
    axs[i].boxplot(data[var].dropna(), vert=False)
    axs[i].set_title(var)

plt.tight_layout()
plt.show()

In [ ]:
#Matriz de Correlacion
#Verifiquemos si hay correlacion entre los datos
corr = data.corr(method="pearson").corr()
#Resulta más intuitivo ver la matriz de correlación a través de un gráfico - heatmap de seaborn

plt.figure(figsize=(15,15)) #Defino el tamaño del grafico

#Genero el mapa de calor
sns.heatmap(corr, 
            cbar = True, 
            square = True, 
            annot=True,
            fmt= '.2f',
            annot_kws={'size': 15},
            cmap= 'coolwarm') 

#Roto los ejes para que queden inclinados
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()

# Guardar los datos estandarizados

In [ ]:
from paths import data_interim_dir
df.to_csv(data_interim_dir("datos_procesados.csv"), index=False)